2. 레이블링
  - 얼마나 뒤의 금리변동을 예측할 것인지 t + 1 기준 선정
  - (선택사항) 혹시라도 예측까지 파이프라인을 구축한다면, 다양한 기간으로 테스트 해봐도 좋습니다.
3. 모델링
  - Market Approach
  - Lexical Approach

# 1. 전처리

In [18]:
# ngram 생성하기

import os
import re
from ekonlpy.sentiment import MPCK
import pandas as pd

TXT_DIR = "../../01_Crawling/financecrawler/TXT"
PDF_DIR = "../../01_Crawling/financecrawler/PDF"
data = pd.read_csv('../../01_Crawling/financecrawler/new.csv')
df = pd.DataFrame(columns=['date', 'content', 'scores', 'ngrams'])

text_list = os.listdir(TXT_DIR)
pdf_list = os.listdir(PDF_DIR)

ngram_list = []
scores = []
content = []
dates = []

for i in range(len(text_list)):
    # 텍스트 불러오기
    text_file = open(TXT_DIR + f"/{text_list[i]}", 'r', encoding='utf-8-sig')
    text = ''
    for line in text_file:
        new_line = line.strip()
        if len(new_line) == 0:
            continue
        else:
            text += new_line

    re_text = re.sub('[^ ㄱ-ㅣ가-힣]+', '', text).lstrip().rstrip()
    
    content.append(re_text)
    dates.append(data['date'].loc[i])

    # MPCK 객체 생성
    mpck = MPCK()

    tokens = mpck.tokenize(re_text)
    ngrams = mpck.ngramize(tokens)
    score = mpck.classify(tokens+ngrams, intensity_cutoff=1.5)

    scores.append(score)
    ngram_list.append(ngrams)

In [5]:
dates

['11.01.03', '11.01.03', '11.01.03', '11.01.04', '11.01.05']

In [34]:
df['date'] = dates
df['date'] = pd.to_datetime(df['date'], yearfirst=True)
df['content'] = content
df['scores'] = scores
df['ngrams'] = ngram_list

In [35]:
df.to_csv('bondreport.csv', mode='w',encoding='utf-8-sig')

# 2. 레이블링

# 3. 모델링